In [1]:
import warnings
warnings.filterwarnings('ignore')

import requests
from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
url = "https://results.eci.gov.in/PcResultGenJune2024/index.htm"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
# Find tables with election results
tables = soup.find_all('table')

# Extract headers
headers = tables[0].find_all('th')
headers = [ele.text.strip() for ele in headers[:4]]  # Adjust if there are more headers

In [4]:
# Extract data into a DataFrame
data = []
for table in tables:
    rows = table.find_all('tr')
    for row in rows[1:]:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        if len(cols) == len(headers):  # Ensure the row matches header length
            data.append(cols)

df = pd.DataFrame(data, columns=headers)

In [5]:
df.head()

,Party,Won,Leading,Total
0,Bharatiya Janata Party - BJP,240,0,240
1,Indian National Congress - INC,99,0,99
2,Samajwadi Party - SP,37,0,37
3,All India Trinamool Congress - AITC,29,0,29
4,Dravida Munnetra Kazhagam - DMK,22,0,22


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Party    42 non-null     object
 1   Won      42 non-null     object
 2   Leading  42 non-null     object
 3   Total    42 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


In [7]:
# Convert 'Won' and 'Leading' columns to numeric
df['Won'] = pd.to_numeric(df['Won'], errors='coerce')
df['Leading'] = pd.to_numeric(df['Leading'], errors='coerce')
df['Total'] = pd.to_numeric(df['Total'], errors='coerce')

# Sort DataFrame by 'Won' column in descending order
df = df.sort_values(by='Won', ascending=False)

# Select top 7 parties and club the rest as "Others"
top_parties = df.head(7)
others = pd.DataFrame([['Others', df['Won'][7:].sum(), df['Leading'][7:].sum(), df['Total'][7:].sum()]], columns=headers)

# Combine top parties and "Others"
combined_df = pd.concat([top_parties, others])

In [8]:
# Colors for each party
colors = ['#FF9933', '#3399FF', '#FF3333', '#99CCFF', '#33CC33', '#FFCC33', '#3333FF', '#CCCCCC']

# Create donut chart
fig = go.Figure(data=[go.Pie(labels=combined_df['Party'], 
                             values=combined_df['Won'], 
                             hole=.4,
                             marker_colors=colors,
                             textinfo='label+percent',
                             insidetextorientation='radial',
                             sort=False)])

fig.update_layout(
    title_text='Election Results - Top 7 Parties and Others',
    annotations=[dict(text=f'{combined_df["Won"].sum()}/543', 
                      x=0.5, y=0.5, font_size=24, showarrow=False)],
    showlegend=True
)

fig.show()

In [9]:
colors = px.colors.sequential.Oranges[::-1][:7] + ['#CCCCCC'] 

# Bar Chart: Number of seats won by each party
fig_bar = px.bar(combined_df, x='Party', y='Won', title='Number of Seats Won by Each Party', 
                 labels={'Won': 'Seats Won', 'Party': 'Party'},
                 color='Won', color_continuous_scale=colors)

fig_bar.show()

In [10]:
fig_treemap = px.treemap(combined_df, path=['Party'], values='Won', title='Proportion of Seats Won by Each Party',
                         color='Won', color_continuous_scale=px.colors.sequential.Oranges[::-1],
                         labels={'Won': 'Seats Won'})

fig_treemap.show()

In [11]:
# Key Insights
total_seats = df['Total'].astype(int).sum()
winning_party = df.loc[df['Won'].astype(int).idxmax()]['Party']
top_5_parties = df[['Party', 'Won']].sort_values(by='Won', ascending=False).head(5)
total_won_seats = df['Won'].astype(int).sum()
total_leading_seats = df['Leading'].astype(int).sum()
leading_party = df.loc[df['Leading'].astype(int).idxmax()]['Party']
least_5_won_parties = df[['Party', 'Won']].sort_values(by='Won', ascending=True).head(5)
winning_party_seats = df[df['Party'] == winning_party]['Won'].astype(int).sum()
winning_party_percentage = (winning_party_seats / total_seats) * 100
parties_with_seats = df[df['Won'].astype(int) > 0]['Party'].nunique()
total_parties = df['Party'].nunique()

formatted_text = (
    "10 Key Insights:-\n\n"
    "1. Total Seats: {}\n\n"
    "2. Winning Party: {}\n\n"
    "3. Top 5 Parties by Seats Won:\n{}\n\n"
    "4. Total Seats Won: {}\n\n"
    "5. Total Seats Leading: {}\n\n"
    "6. Party with Most Leading Seats: {}\n\n"
    "7. Least 5 Parties by Seats Won:\n{}\n\n"
    "8. Winning Party Percentage of Total Seats: {:.2f}%\n\n"
    "9. Number of Parties with At Least One Seat: {}\n\n"
    "10. Total Number of Parties Participated: {}"
).format(
    total_seats,
    winning_party,
    top_5_parties.to_string(index=False),
    total_won_seats,
    total_leading_seats,
    leading_party,
    least_5_won_parties.to_string(index=False),
    winning_party_percentage,
    parties_with_seats,
    total_parties
)

print(formatted_text)

10 Key Insights:-

1. Total Seats: 543

2. Winning Party: Bharatiya Janata Party - BJP

3. Top 5 Parties by Seats Won:
                              Party  Won
       Bharatiya Janata Party - BJP  240
     Indian National Congress - INC   99
               Samajwadi Party - SP   37
All India Trinamool Congress - AITC   29
    Dravida Munnetra Kazhagam - DMK   22

4. Total Seats Won: 543

5. Total Seats Leading: 0

6. Party with Most Leading Seats: Bharatiya Janata Party - BJP

7. Least 5 Parties by Seats Won:
                                         Party  Won
       Hindustani Awam Morcha (Secular) - HAMS    1
             Rashtriya Loktantrik Party - RLTP    1
All India Majlis-E-Ittehadul Muslimeen - AIMIM    1
                            AJSU Party - AJSUP    1
                    Apna Dal (Soneylal) - ADAL    1

8. Winning Party Percentage of Total Seats: 44.20%

9. Number of Parties with At Least One Seat: 42

10. Total Number of Parties Participated: 42


In [12]:
# Define party alliances
indi_alliance = [
    'Indian National Congress - INC',
    'Samajwadi Party - SP',
    'Dravida Munnetra Kazhagam - DMK',
    'Nationalist Congress Party – Sharadchandra Pawar',
    'Rashtriya Janata Dal - RJD',
    'Communist Party of India (Marxist) - CPI(M)',
    'Communist Party of India (Marxist-Leninist) - CPIML',
    'Communist Party of India - CPI',
    'Jharkhand Mukti Morcha - JMM',
    'Indian Union Muslim League - IUML',
    'Viduthalai Chiruthaigal Katchi - VCK',
    'Kerala Congress - KEC',
    'Rashtriya Lok Dal - RLD',
    'Jammu & Kashmir National Conference - JKN',
    'Revolutionary Socialist Party - RSP',
    'Marumalarchi Dravida Munnetra Kazhagam - MDMK'
]

nda_alliance = [
    'Bharatiya Janata Party - BJP',
    'All India Trinamool Congress - AITC',
    'Telugu Desam - TDP',
    'Janata Dal (United) - JD(U)',
    'Shiv Sena (Uddhav Balasaheb Thackrey) - SHSUBT',
    'Shiv Sena - SHS',
    'Lok Janshakti Party(Ram Vilas) - LJPRV',
    'Yuvajana Sramika Rythu Congress Party - YSRCP',
    'Janasena Party - JnP',
    'Janata Dal (Secular) - JD(S)',
    'Asom Gana Parishad - AGP',
    'Hindustani Awam Morcha (Secular) - HAMS',
    'Shiromani Akali Dal - SAD',
    'Apna Dal (Soneylal) - ADAL',
    'AJSU Party - AJSUP',
]

# Create new DataFrame for alliances
alliances = {
    'Alliance': ['INDI Alliance', 'NDA Alliance', 'Others'],
    'Seats Won': [0, 0, 0]
}

for index, row in df.iterrows():
    party = row['Party']
    if party in indi_alliance:
        alliances['Seats Won'][0] += int(row['Won'])
    elif party in nda_alliance:
        alliances['Seats Won'][1] += int(row['Won'])
    else:
        alliances['Seats Won'][2] += int(row['Won'])

alliances_df = pd.DataFrame(alliances)

In [13]:
# Colors for alliances
colors = ['#00A1DE', '#FF9933', '#CCCCCC']

# Create donut chart
fig = go.Figure(data=[go.Pie(labels=alliances_df['Alliance'], 
                             values=alliances_df['Seats Won'], 
                             hole=.4,
                             marker_colors=colors,
                             textinfo='label+percent',
                             insidetextorientation='radial',
                             sort=False)])

fig.update_layout(
    title_text='Election Results - INDI Alliance vs NDA Alliance',
    annotations=[dict(text=f'{sum(alliances_df["Seats Won"])}/543', 
                      x=0.5, y=0.5, font_size=24, showarrow=False)],
    showlegend=True
)

fig.show()